In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import metrics

from tensorflow.keras.utils import get_file

In [3]:
# Read the data
import glob,os
path=r'../Dataset/'
file=glob.glob(os.path.join(path, "data*.csv"))
print(file)
dl= []
for f in file:
 dl.append(pd.read_csv(f,header=[0],index_col=None))
df=pd.concat(dl)

['../Dataset\\data1.csv', '../Dataset\\data10.csv', '../Dataset\\data11.csv', '../Dataset\\data12.csv', '../Dataset\\data13.csv', '../Dataset\\data14.csv', '../Dataset\\data15.csv', '../Dataset\\data2.csv', '../Dataset\\data3.csv', '../Dataset\\data4.csv', '../Dataset\\data5.csv', '../Dataset\\data6.csv', '../Dataset\\data7.csv', '../Dataset\\data8.csv', '../Dataset\\data9.csv']


In [4]:
# label Encoding
df['marker'] = df['marker'].replace(['Natural'],0)
df['marker'] = df['marker'].replace(['Attack'],1)
df['marker'].value_counts()

1    55663
0    22714
Name: marker, dtype: int64

In [5]:
# Separate target from predictors
y = df.marker
x = df.drop(['marker'], axis=1)

In [6]:
values = x
# np.all(np.isfinite(values))
values.replace([np.inf, -np.inf], np.nan, inplace=True)

In [7]:
#normalization
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_normalized=scaler.fit_transform(values)
X_normalized

array([[0.6955546 , 0.84220973, 0.36229508, ..., 0.        , 0.        ,
        0.        ],
       [0.70469051, 0.85941118, 0.3713831 , ..., 0.        , 0.        ,
        0.        ],
       [0.70481784, 0.85957658, 0.37151042, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.33600726, 0.86470394, 0.00275346, ..., 0.        , 0.        ,
        0.        ],
       [0.33587993, 0.86486934, 0.00262613, ..., 0.        , 0.        ,
        0.        ],
       [0.33552977, 0.86520013, 0.0022919 , ..., 0.        , 0.        ,
        0.        ]])

In [8]:
from sklearn.model_selection import train_test_split
# Divide data into training and validation subsets
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X_normalized, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

In [9]:
# reshape input to be 3D [samples, timesteps, features]
X_train_full = X_train_full.reshape((X_train_full.shape[0], 1, X_train_full.shape[1]))
X_valid_full = X_valid_full.reshape((X_valid_full.shape[0], 1, X_valid_full.shape[1]))
print(X_train_full.shape, y_train.shape, X_valid_full.shape, y_valid.shape) 

(62701, 1, 128) (62701,) (15676, 1, 128) (15676,)


In [12]:
data_dim = 128
timesteps = 1
num_classes = 1

# expected input data shape: (batch_size, timesteps, data_dim)
model = keras.Sequential()
model.add(keras.layers.LSTM(32, return_sequences=True,   input_shape=(timesteps, data_dim)))   
# model.add(keras.layers.LSTM(32, return_sequences=True))  
# model.add(keras.layers.LSTM(32))  
model.add(keras.layers.Dense(1, activation='softmax'))

model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
model.summary()


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 1, 32)             20608     
_________________________________________________________________
dense_2 (Dense)              (None, 1, 1)              33        
Total params: 20,641
Trainable params: 20,641
Non-trainable params: 0
_________________________________________________________________


In [13]:
model.fit(X_train_full, y_train,  batch_size= 60, epochs= 10)

Epoch 1/10
1046/1046 [==============================] - 1s 1ms/step - loss: nan - accuracy: 0.2902
Epoch 2/10
1046/1046 [==============================] - 2s 2ms/step - loss: nan - accuracy: 0.2902
Epoch 3/10
1046/1046 [==============================] - 2s 1ms/step - loss: nan - accuracy: 0.2902
Epoch 4/10
1046/1046 [==============================] - 2s 2ms/step - loss: nan - accuracy: 0.2902
Epoch 5/10
1046/1046 [==============================] - 2s 2ms/step - loss: nan - accuracy: 0.2902
Epoch 6/10
1046/1046 [==============================] - 1s 1ms/step - loss: nan - accuracy: 0.2902
Epoch 7/10
1046/1046 [==============================] - 1s 1ms/step - loss: nan - accuracy: 0.2902
Epoch 8/10
1046/1046 [==============================] - 1s 1ms/step - loss: nan - accuracy: 0.2902
Epoch 9/10
1046/1046 [==============================] - 1s 1ms/step - loss: nan - accuracy: 0.2902
Epoch 10/10
1046/1046 [==============================] - 1s 1ms/step - loss: nan - accuracy: 0.2902


In [14]:
score = model.evaluate(X_valid_full, y_valid, batch_size=60)

262/262 [==============================] - 0s 598us/step - loss: nan - accuracy: 0.2881
